In [1]:
# coding: utf-8
import warnings
warnings.filterwarnings('ignore')
import time
import os
import numpy as np
import keras
from keras import metrics
from keras.utils import to_categorical
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, Flatten, Conv2D, Conv1D, MaxPooling1D, MaxPooling2D,Activation
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.layers.embeddings import Embedding
from datetime import datetime
from gensim.models import word2vec
from collections import *
import pandas as pd
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
def text_to_index(corpus):
    new_corpus = []
    for doc in corpus:
        new_doc = []
        for word in doc:
            try:
                new_doc.append(word2idx[word])
            except:
                new_doc.append(0)
#         new_doc_arr = np.array(new_doc).reshape(1, max_doc_word_length)
        new_doc_arr = np.array(new_doc)
        new_corpus.append( new_doc_arr)
    return np.array(new_corpus)

In [7]:
# get data
# get texts data
category2idx = {'AllTogether': 0, 'Baseball': 1, 'Boy-Girl': 2, 'C_chat':  3, 'CVS': 4,
                  'GameSale': 5, 'GetMarry': 6, 'Lifeismoney': 7, 'LoL': 8, 'MH': 9, 'MLB': 10, 'Mobilecomm': 11, 
                'movie': 12,'MuscleBeach':  13, 'NBA': 14,  'SENIORHIGH': 15, 'Stock': 16, 
                'Tennis': 17, 'Tos': 18, 'WomenTalk': 19}

train_df_sample = pd.read_pickle('train_reduce.pkl').sample(frac=1, random_state=123)
train_texts = train_df_sample.ari_title
label_list = train_df_sample.price

test_pickle_df = pd.read_pickle('train_reduce.pkl')
test_texts = test_pickle_df["ari_title"].values

train_texts_list = []
for text in train_texts:
    train_texts_list.append(text[0])
    
texts_list = []
for text in train_texts_list:
    texts_list.append(text)
    
for text in test_texts:
    texts_list.append(text)

In [8]:
# get word embedding vector
answer = word2vec.Word2Vec.load("word2vec.model")
word_vectors = answer.wv
wvv = word_vectors.vocab
wvv_keys = wvv.keys()
wvv_keys_list = list(wvv_keys)

vocab_num = len(wvv.items()) + 1

vocab_list = [(word, word_vectors[word]) for word, _ in wvv.items()]

In [9]:
del word_vectors, wvv, train_texts_list, answer

In [10]:
word_vec_len = 250
embedding_matrix = np.zeros((vocab_num , word_vec_len))
word2idx = {}

for i, vocab in enumerate(vocab_list):
    word, vec = vocab
    embedding_matrix[i + 1] = vec
    word2idx[word] = i + 1

In [11]:
embedding_matrix.shape

(46, 250)

In [13]:
embedding_layer = Embedding( input_dim= embedding_matrix.shape[0],output_dim= 250, weights=[embedding_matrix], 
                            input_length = 200,trainable=False)
# model = getModel(embedding_layer, image_width, image_height, input_channel)
model = Sequential()
model.add(embedding_layer )
model.add(Conv1D(256, 3,padding = 'same', ))
model.add(Flatten())
model.add(Dense(2, activation='relu'))
model.compile(optimizer='adam',loss='mae')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 200, 250)          11500     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 200, 256)          192256    
_________________________________________________________________
flatten_1 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 102402    
Total params: 306,158
Trainable params: 294,658
Non-trainable params: 11,500
_________________________________________________________________


In [21]:
max_doc_word_length = 200

x_trains_texts = train_df_sample.ari_title.append(test_pickle_df.ari_title)
X_train_texts = text_to_index(x_trains_texts)
X_train = pad_sequences(X_train_texts, maxlen= max_doc_word_length)

print(x_trains_texts.shape)
print(len(label_list))

Y_label_list = np.zeros((6577, 2))
for ids in range(0, 6577):
    Y_label_list[ids]  = label_list[ids] 
    Y_label_list[ids]  = label_list[ids] 
print(Y_label_list.shape)

(13154,)
6577
(6577, 2)


In [ ]:
history = model.fit(x = X_train[0:6577], 
                    y = Y_label_list, 
                    batch_size= 100,
                    epochs = 10, verbose = 1)

Epoch 1/10
6577/6577 [==============================] - 21s 3ms/step - loss: 0.5289
Epoch 2/10
6577/6577 [==============================] - 21s 3ms/step - loss: 0.5373
Epoch 3/10
3700/6577 [===============>..............] - ETA: 9s - loss: 0.5351

In [ ]:
# np_loss_history = np.array(history.history['loss'])
# np.savetxt("loss_history.txt", np_loss_history, delimiter=",")
loss_history = np.loadtxt("loss_history.txt")
fig = plt.figure()
ax = plt.axes()
x = np.linspace(0, 1, loss_history.shape[0])

plt.plot(x, loss_history, '-g');  # dotted red


In [ ]:
# model.save('my_model.h5') 

In [ ]:
# evaluate the model
loss_accuracy = model.evaluate(X_train[0:100], Y_label_list[0:100], verbose=1)
print(type(loss_accuracy), loss_accuracy)

test_sequences1 = X_train[6477:6577]

predict_res = model.predict(test_sequences1, batch_size= 100, verbose=0)

final_res = []
for pre_res in predict_res:
    final_res.append(pre_res)
# print(final_res)

In [ ]:
print(predict_res[0])

In [ ]:
# result_txt = "result" + str(datetime.now()).split()[1] + ".txt"
print(len(final_res))
result_txt = "local_result001" + ".txt"
ids = 0
with open(result_txt, 'w') as out:
    out.write("id,good,bad" + '\n')
    for value in final_res:
        out.write(str(ids) + "," + str(int (round(value[0]))) + "," + str(int (round(value[0]))) + '\n')
        ids += 1